# Planning

1) Faço o retrieve da thread para conseguir organizar melhor o prompt.
    - Para pegar o ThreadID correto irei filtrar pelo título, origem e conteúdo. Se der match nos 3, então pego o ID.
    - Fazer um retrieve das mensagens do usuário apenas, sendo a mensagem [0] a última.
2) Criar um prompt
    - Monto um prompt para extrair as perguntas usando apenas os emails que vieram do usuário.
    - As perguntas devem ser claras e completas.
3) Busca no Vertex Search
    - Compila as resposta e manda por email.
4) Caso o usuário escreva a palavra "human" ou "agent" ou "talk", vou redirecionar para um humano responder.
    - Aqui precisamos criar um draft.

# Step 1 - Retrieve or create Vertex AI Search conversation

In [1]:
import base64
import utils_palm
import numpy as np
import utils_vertex_vector

In [2]:
import json
from email.message import EmailMessage
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from google.cloud import discoveryengine_v1beta as discoveryengine
from google.cloud import firestore
from google.oauth2.credentials import Credentials
from proto import Message
from vertexai.preview.language_models import TextGenerationModel

import tomllib

with open("config.toml", "rb") as f:
    config = tomllib.load(f)

db = firestore.Client()

project_id = config["global"]["project_id"]
location = config["global"]["location"]
datastore_location = config["global"]["datastore_location"]
serving_config = config["global"]["serving_config"]
images_bucket_name = config["global"]["images_bucket_name"]
email_datastore_id = config["salesforce"]["email_datastore_id"]
project_number = config["global"]["project_number"]

# Workspace integration
CREDENTIALS = Credentials.from_authorized_user_file(
    filename="token.json", 
    scopes=['https://www.googleapis.com/auth/gmail.modify'])
gmail_services = build('gmail', 'v1', credentials=CREDENTIALS)

model = TextGenerationModel.from_pretrained("text-bison@latest")

In [3]:
# temp
email_content = """Hi Cymbal Support,"""
subject = "Are gaming chairs ergonomic?"
email_address = "renatoleite@google.com"

In [ ]:
prompt_email = """<instructions>
Extract questions or inquiries that are being asked in the conversation below. If you don't find any questions or inquiries, respond with an empty list.
Provide the output in the JSON format below.
</instructions>
<conversation>
{}
</conversation>
<JSONformat>
{{
    "questions": [
        "Question or inquiry 1",
        "Question or inquiry 2",
        "Question or inquiry 3",
        "Question or inquiry 4",
    ]
}}
</JSONformat>
<output>"""

In [ ]:
converse_client = discoveryengine.ConversationalSearchServiceClient()

In [ ]:
email = "renatoleite@google.com"
case_id = "00001063"

In [ ]:
document = db.collection("salesforce_cases").document(case_id)

if document.get().exists:
    conversation_resource = document.get().to_dict()["conversation_resource"]
    email_thread_id = document.get().to_dict()["email_thread_id"]
    request = discoveryengine.GetConversationRequest(name=conversation_resource)
    conversation = converse_client.get_conversation(request=request)
else:
    converse_request = discoveryengine.CreateConversationRequest(
        parent = (f"projects/{project_id}/"
                    f"locations/{datastore_location}/"
                    "collections/default_collection/"
                    f"dataStores/{email_datastore_id}"),
        conversation=discoveryengine.Conversation(
            user_pseudo_id=email
        )
    )
    conversation = converse_client.create_conversation(
                request=converse_request)

    # Retrieve thread_id from Gmail
    q = f'from:{email_address} label:inbox ' \
        f'subject:"{subject}" "{email_content}"'
    emails = gmail_services.users().threads().list(
        userId="renatoleite@1987984870407.altostrat.com",
        maxResults=1,
        q=q
    ).execute()

    if emails:
        email_thread_id = emails["threads"][0]["id"]
        document.set({
            "conversation_resource": conversation.name,
            "email_thread_id": email_thread_id
        })
    else:
        raise # TODO: include raise HTTP

try:
    questions = model.predict(
        prompt=prompt_email.format(email_content)
    ).text
    questions_dict = json.loads(questions)

except Exception as e:
    raise # TODO: include raise HTTP



In [ ]:
conversation

# Step 2 - Retrieve email_thread_id logic

In [ ]:
email_content = """"""
subject = "Are gaming chairs ergonomic?"
email_address = "renatoleite@google.com"

q = f'from:renatoleite@google.com label:inbox subject:"Are gaming chairs ergonomic?" "{email_content}"'

In [ ]:
emails = gmail_services.users().threads().list(
    userId="renatoleite@1987984870407.altostrat.com",
    maxResults=1,
    q=q
).execute()

In [ ]:
email_thread_id = emails["threads"][0]["id"]

In [ ]:
email_thread_id

In [ ]:
# Check if there are attachements (JPEG or PNG) in the email

In [ ]:
email_message = gmail_services.users().threads().get(
    userId="renatoleite@1987984870407.altostrat.com",
    id=email_thread_id
).execute()

In [ ]:
# Get attachmentId from message
# TODO: mudar apenas para a última mensagem

attachments = set()

email_message_dict = email_message["messages"][-1]
email_message_id = email_message_dict["id"]

for part_level_0 in email_message_dict["payload"]["parts"]:
    if (part_level_0["mimeType"] == "image/png" or 
        part_level_0["mimeType"] == "image/jpg"):
        attachments.add(part_level_0["body"]["attachmentId"])
    elif "multipart" in part_level_0["mimeType"]:
        for part_level_1 in part_level_0["parts"]:
            if (part_level_1["mimeType"] == "image/png" or 
                part_level_1["mimeType"] == "image/jpg"):
                attachments.add(part_level_1["body"]["attachmentId"])

In [ ]:
embedding = utils_palm.EmbeddingPredictionClient(project=project_id)

In [ ]:
prompt_multimodal = """<instructions>
Below is an email from a customer to a support center with an attached image.
Extract questions or inquiries that are being asked in the conversation below.
Separate the questions or inquiries that are related to the attached image and the questions or inquiries that are not related to the attached image.
If you don't find any questions or inquiries, respond with an empty list.
Provide the output in the JSON format below.
</instructions>
<conversation>
{}
<conversation>
<JSONformat>
{{
    "questions_inquiries_not_related_to_image": [
        "Question or inquiry 1, not related to the attached image?",
        "Question or inquiry 2, not related to the attached image?",
        "Question or inquiry 3, not related to the attached image?",
    ],
    "questions_inquiries_related_to_image": [
        "Question or inquiry 1, related to the image?",
        "Question or inquiry 2, related to the image?",
        "Question or inquiry 3, related to the image?",
    ]
}}
</JSONformat>
<output>"""

In [ ]:
email_content = """Hi Cymbal Support,
I'm interested in purchasing a gaming chair from your store and I have a few questions.
First, are your gaming chairs ergonomic?
Second, do you have any gaming chairs that are made of nylon?
Show me products similar to the attached image. I prefer blue chairs.
Thanks for your help,"""

In [ ]:
# Extract embedding for each email and sum them
images_embeddings = []
for attachment in list(attachments):
    email_attachment = gmail_services.users().messages().attachments().get(
        userId="renatoleite@1987984870407.altostrat.com",
        messageId=email_message_id,
        id=attachment
    ).execute()
    
    # Replace the characters in the base64 string
    image_bytes = email_attachment["data"].replace('_', '/').replace('-', '+').replace(' ', '')
    image_bytes = base64.b64decode(image_bytes)

    images_embeddings.append(
        embedding.get_embedding(
            image_bytes=image_bytes
        ).image_embedding
    )

# Check for questions / inquiries
multimodal_questions = json.loads(model.predict(
    prompt=prompt_multimodal.format(email_content)
).text)

image_embedding = list(np.sum(np.array(images_embeddings), axis=0))

text_embeddings = []
if multimodal_questions.get("questions_inquiries_related_to_image", ""):
    for q in multimodal_questions["questions_inquiries_related_to_image"]:
        text_embeddings.append(
            embedding.get_embedding(text=q).text_embedding
        )

text_embedding = list(np.sum(np.array(text_embeddings), axis=0))

multimodal_embedding = utils_palm.reduce_embedding_dimesion(
    vector_text=text_embedding,
    vector_image=image_embedding
)

utils_vertex_vector.find_neighbor(
    
)

# Step 3 - Prompt to extract questions

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

model = TextGenerationModel.from_pretrained("text-bison@latest")

In [ ]:
email_content = """Hi Cymbal Support,


I'm interested in purchasing a gaming chair from your store and I have a few questions.


First, are your gaming chairs ergonomic? I'm looking for a chair that will support my back and help me avoid pain while gaming.


Second, do you have any gaming chairs that are made of nylon? I'm allergic to some synthetic materials and I need to make sure that the chair I choose is safe for me to use.


Thanks for your help,"""

In [ ]:
questions = model.predict(
    prompt=prompt_email.format(email_content)
).text

In [ ]:
import json
questions_dict = json.loads(questions)

In [ ]:
questions_dict

# Env setup

In [2]:
from email.message import EmailMessage
from email.mime.text import MIMEText

from googleapiclient.discovery import build

from google.oauth2.credentials import Credentials
from proto import Message

import tomllib

In [3]:
# Load configuration file
with open("config.toml", "rb") as f:
    config = tomllib.load(f)
project_id = config["global"]["project_id"]
location = config["global"]["location"]

In [4]:
# Workspace integration
CREDENTIALS = Credentials.from_authorized_user_file(
    filename="token.json", 
    scopes=['https://www.googleapis.com/auth/gmail.modify'])
gmail_services = build('gmail', 'v1', credentials=CREDENTIALS)

# List messages

In [4]:
email_body = """Are gaming chairs ergonomic?"""

# q = "from:renatoleite@google.com subject:Help with gaming chairs"
q = f"from:renatoleite@google.com subject:'test' {email_body}"

emails = gmail_services.users().messages().list(
    userId="renatoleite@1987984870407.altostrat.com",
    q=q
).execute()

In [5]:
emails

{'messages': [{'id': '18b7187674715a55', 'threadId': '18b717b364707ec2'},
  {'id': '18b717bef399475b', 'threadId': '18b717b364707ec2'},
  {'id': '18b717b364707ec2', 'threadId': '18b717b364707ec2'}],
 'resultSizeEstimate': 3}

In [7]:
emails = gmail_services.users().threads().list(
    userId="renatoleite@1987984870407.altostrat.com",
    # maxResults=1,
    q="rfc822msgid:<CA+J_hzNqg9BDzYpJ+Df-kk0cQsKhRGJ-1wCWT9fof4U1cq_s=A@mail.gmail.com>"
).execute()

In [8]:
emails

{'threads': [{'id': '18b717b364707ec2',
   'snippet': 'Are gaming chairs ergonomic? Renato Leite Solutions Architect - Machine Learning +55 11 97368.8868',
   'historyId': '21055'}],
 'resultSizeEstimate': 1}

In [10]:
email_message = gmail_services.users().threads().get(
    userId="renatoleite@1987984870407.altostrat.com",
    id="18b717b364707ec2"
).execute()

In [11]:
# Get thread ID from the first message
for header in email_message["messages"][0]["payload"]["headers"]:
    if header["name"] == "Message-ID":
        email_thread_id = header["value"]

# Get message ID from the last message
for header in email_message["messages"][-1]["payload"]["headers"]:
    if header["name"] == "Message-ID":
        email_message_id = header["value"]

In [12]:
email_thread_id

'<CA+J_hzNqg9BDzYpJ+Df-kk0cQsKhRGJ-1wCWT9fof4U1cq_s=A@mail.gmail.com>'

In [13]:
email_message_id

'<CA+J_hzOrkdgZ4pE12pwi_jyjMJcEDBAx0Sxa8Twn8P+bZR41hQ@mail.gmail.com>'

In [9]:
message["messages"][-1]

{'id': '18b7187674715a55',
 'threadId': '18b717b364707ec2',
 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL'],
 'snippet': 'hi, how are you? Renato Leite Solutions Architect - Machine Learning +55 11 97368.8868 On Fri, Oct 27, 2023 at 11:13 AM Renato Leite &lt;renatoleite@google.com&gt; wrote: Are they made of nylon? Renato',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'renatoleite@1987984870407.altostrat.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6520:516:b0:27b:9d14:7478 with SMTP id g22csp2191817lkv;        Fri, 27 Oct 2023 07:26:21 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:aa7:c70b:0:b0:53e:5957:aa49 with SMTP id i11-20020aa7c70b000000b0053e5957aa49mr2510466edq.19.1698416780906;        Fri, 27 Oct 2023 07:26:20 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1698416780; cv=none;        d=google.com; s=arc-20160816;        b=J0cv4t2cN

In [29]:
for h in message["messages"][0]["payload"]["headers"]:
    if h["name"] == "Message-ID":
        message_id = h["value"]

In [30]:
message_id

'<CA+J_hzNqg9BDzYpJ+Df-kk0cQsKhRGJ-1wCWT9fof4U1cq_s=A@mail.gmail.com>'

# Send message

In [46]:
import base64
# message = EmailMessage()

email_response = """
resposta teste 8
"""

message = MIMEText(email_response, "html")

# message.set_content(email_response)

message['To'] = "renatoleite@google.com"
message['In-Reply-To'] = "<CA+J_hzO1-ZeGkMU3Y4wM7x5oEVEG8ejqMkV0KQK-KcbDhctHVQ@mail.gmail.com>"
message['From'] = 'renatoleite@1987984870407.altostrat.com'
message['Subject'] = "Test 1"
message['References'] = "<CA+J_hzO1-ZeGkMU3Y4wM7x5oEVEG8ejqMkV0KQK-KcbDhctHVQ@mail.gmail.com>"

# encoded message
encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

create_message = {
    'raw': encoded_message,
    'threadId': "18b6dcf5d66b3cff"
}

gmail_services.users().messages().send(
    userId="renatoleite@1987984870407.altostrat.com", 
    body=create_message).execute()

{'id': '18b6dee8318e08e1',
 'threadId': '18b6dcf5d66b3cff',
 'labelIds': ['SENT']}

# Prompt for email

In [ ]:
conversation = """Hi Cymbal Support,
I'm interested in purchasing a gaming chair from your store and I have a few questions.
First, are your gaming chairs ergonomic? I'm looking for a chair that will support my back and help me avoid pain while gaming.
Second, do you have any gaming chairs that are made of nylon? I'm allergic to some synthetic materials and I need to make sure that the chair I choose is safe for me to use.
Thanks for your help,"""

prompt_email = """<instructions>
You have to extract questions that are being asked in the conversation below.
Provide the output in the JSON format below.
</instructions>
<conversation>
{}
<conversation>
<JSONformat>
{
    "questions": [
        "Question 1",
        "Question 2",
        "Question 3",
        "Question 4",
    ]
}
</JSONformat>
<output>"""

# Create or retrieve existing conversation

In [ ]:
from google.cloud import discoveryengine_v1beta as discoveryengine
import tomllib

with open("config.toml", "rb") as f:
    config = tomllib.load(f)

In [ ]:
email_datastore_id = config["salesforce"]["email_datastore_id"]

In [ ]:
converse_client = discoveryengine.ConversationalSearchServiceClient()

In [ ]:
converse_client.list_conversations(
    discoveryengine.ListConversationsRequest(
        parent = f"projects/rl-llm-dev/locations/global/collections/default_collection/dataStores/{email_datastore_id}"
    )
)